In [1]:
from transformers import (
    AutoModelForSequenceClassification,  # 시퀀스 분류 모델
    AutoTokenizer,  # 토크나이저
    DataCollatorWithPadding,  # 패딩 데이터 콜레이터
    set_seed,
    Trainer,  # 트레이너
    TrainingArguments,  # 훈련 설정
)
import os
import torch

In [2]:
# 환경변수 설정 (토크나이저 병렬처리 경고 해결)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 토크나이저 및 모델 설정
CHOSEN_MAX_LENGTH = 128
NUM_CLASSES = 4  # 클래스 개수 (0: 강한부정, 1: 약한부정, 2: 약한긍정, 3: 강한긍정)

# 배치 크기 설정
BATCH_SIZE_TRAIN = 256  # 훈련용
BATCH_SIZE_EVAL = 128  # 평가용

# 훈련 하이퍼파라미터
LEARNING_RATE = 2e-5  # AdamW 최적화기 학습률
NUM_EPOCHS = 5  # 훈련 에포크 수
WARMUP_STEPS = 500  # 학습률 워밍업 스텝
WEIGHT_DECAY = 0.01  # 가중치 감쇠 (정규화)

# GPU 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPU 메모리 정리 및 확인
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU {torch.cuda.device_count()}개 사용 가능: {device}")
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("⚠️  CUDA 사용 불가 - CPU로 훈련 진행")

⚠️  CUDA 사용 불가 - CPU로 훈련 진행


In [1]:
model_name = "klue/roberta-base"

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os

# 1. 토크나이저 로드
# 절대 경로를 사용하여 로컬 모델 로드
print("로컬 경로에서 토크나이저 로딩 중...")
local_model_path = "/data/ephemeral/home/code/klue-roberta-base-local"

# 경로가 존재하는지 확인
if not os.path.exists(local_model_path):
    print(f"❌ 경로가 존재하지 않습니다: {local_model_path}")
else:
    print(f"✅ 경로 확인됨: {local_model_path}")

tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# 2. 모델 로드
# 마찬가지로 로컬 경로(local_model_path)를 사용
print("로컬 경로에서 모델 로딩 중...")
model = AutoModelForSequenceClassification.from_pretrained(
    local_model_path,
    num_labels=7  # 예: KLUE-TC 감성 분석 클래스 개수
)

print("✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!")

# --- 이제 평소처럼 모델을 사용할 수 있습니다 ---
inputs = tokenizer("이 영화 정말 재미있네요!", return_tensors="pt")
outputs = model(**inputs)
print(outputs.logits)

로컬 경로에서 토크나이저 로딩 중...
✅ 경로 확인됨: /data/ephemeral/home/code/klue-roberta-base-local
로컬 경로에서 모델 로딩 중...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /data/ephemeral/home/code/klue-roberta-base-local and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!
tensor([[ 0.0657,  0.0637, -0.1250,  0.0990,  0.0496, -0.0189, -0.2265]],
       grad_fn=<AddmmBackward0>)


In [3]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

True

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb 사용: True
랜덤 시드: 42


In [4]:
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # 1. 태스크 타입 명시
    r=16,
    lora_alpha=32,
    target_modules=["query", "key", "value"],  # 2. 타겟 모듈 명시
    lora_dropout=0.1,
    bias="none",
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 1,480,711 || all params: 112,104,206 || trainable%: 1.3208


In [5]:
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(32000, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [ ]:
# 모델 저장 여부 및 wandb 사용 여부 설정
SAVE_MODEL = False
USE_WANDB = True

print("\n" + "=" * 50)
print("모델 훈련 시작")
print("=" * 50)

# 훈련 파라미터 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE_TRAIN,
    per_device_eval_batch_size=BATCH_SIZE_EVAL,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    learning_rate=LEARNING_RATE,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch" if SAVE_MODEL else "no",
    load_best_model_at_end=SAVE_MODEL,
    metric_for_best_model="accuracy" if SAVE_MODEL else None,
    greater_is_better=True,
    save_total_limit=2 if SAVE_MODEL else 0,
    report_to="wandb" if USE_WANDB else "none",  # wandb 로깅 조건부 활성화
    run_name="bert-movie-review-classification" if USE_WANDB else None,
    seed=RANDOM_STATE,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    remove_unused_columns=False,
    push_to_hub=False,
    gradient_accumulation_steps=1,
    logging_first_step=True,
    save_safetensors=SAVE_MODEL,
    load_best_model_at_end=True,
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# 훈련 정보 출력
print(f"훈련 샘플: {len(train_dataset):,}개")
print(f"검증 샘플: {len(val_dataset):,}개")
print(f"훈련 에포크: {training_args.num_train_epochs}회")
print(f"배치 크기: {BATCH_SIZE_TRAIN} (훈련) / {BATCH_SIZE_EVAL} (검증)")
print(f"학습률: {LEARNING_RATE}")
print(f"시드값: {RANDOM_STATE}")
print(f"디바이스: {device}")
print(f"wandb 사용: {USE_WANDB}")

# 훈련 실행
try:
    training_results = trainer.train()
    print("\n훈련 완료")
    print(f"최종 훈련 손실: {training_results.training_loss:.4f}")

    # 훈련 로그 정보 출력
    if hasattr(training_results, "log_history"):
        print(f"총 훈련 스텝: {training_results.global_step}")

except KeyboardInterrupt:
    print("\n사용자에 의해 훈련이 중단되었습니다.")
    raise
except Exception as e:
    print(f"\n훈련 중 오류 발생: {str(e)}")
    raise